In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd

import seaborn as sns

import numpy as np
from sklearn.mixture import GaussianMixture
from scipy.stats import multivariate_normal

import numpy as np
from scipy.interpolate import griddata
from scipy.integrate import quad, trapezoid
from scipy.interpolate import CubicSpline


import matplotlib.pylab as plt
from scipy import stats
from matplotlib import cm
from matplotlib.ticker import LinearLocator

from scipy.interpolate import LinearNDInterpolator




# Inspect one of the files to see how things work

In [ ]:
# For example

df = pd.read_csv('muonSteps_100000_GeV.csv')

df

# 'event' is one muon through the rock
# 'track' 1 is always the muon. Sometimes, if there are secondary particles, they have different track numbers

In [ ]:
# Look at the first event
n = 1

#filter = (df['track']==1) & (df['event']==n)
filter = (df['event']==n)

x =  df[filter]['z(m)']
y = df[filter]['E(GeV)']
plt.plot(x,y)

df[filter]

In [ ]:
# Look at the first 100 events in the file

zs = []
plt.figure()
nmax = 100
for n in range(0,nmax):

    #filter = (df['track']==1) & (df['event']==n)
    filter = (df['event']==n)
    
    x =  df[filter]['z(m)']
    y = df[filter]['E(GeV)']
    plt.plot(x,y, label=f'{n}')
    plt.xlabel('z position (meters)')
    plt.ylabel('Muon energy (GeV)')



In [ ]:
# Histogram of all y-values for all muons/events
df['y(m)'].hist(range=(-5, 5), bins=200)

# More diagnostics for a single file

In [ ]:
# Change this variable to read in different files
e_initial = 50000


df = pd.read_csv(f'muonSteps_{e_initial:d}_GeV.csv')

zs = []
xs = []
ys = []
rs = []

# Some of the simulations did not run to completion so manually
# set the number of successful events to run over in the file
nmax = 1000
if e_initial == 300000:
    nmax = 504
elif e_initial == 400000:
    nmax = 183
elif e_initial == 500000:
    nmax = 78

for n in range(0,nmax):

    #print(n)
    filter = (df['track']==1) & (df['event']==n)
    #filter = (df['event']==n)
    
    x = df[filter]['x(m)'].iloc[-1]
    y = df[filter]['y(m)'].iloc[-1]
    z = df[filter]['z(m)'].iloc[-1]
    zs.append(z)
    ys.append(y)
    xs.append(x)
    r = np.sqrt(x**2 + y**2)
    rs.append(r)

plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.hist(zs, bins=50);
plt.xlabel('z (m)')
plt.title(f'Initial muon energy: {int(e_initial/1000):d} (TeV)')


plt.subplot(1,2,2)
plt.hist(rs, bins=50);
plt.xlabel('r (m)')
plt.title(f'Initial muon energy: {int(e_initial/1000):d} (TeV)')

plt.savefig(f'energy_and_transverse_displacement_{e_initial:d}_GeV.png')



plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.hist(xs, bins=50);
plt.xlabel('x (m)')

plt.subplot(1,2,2)
plt.hist(ys, bins=50);
plt.xlabel('y (m)')

In [ ]:
df

# Calculating probabilities

Run this code to generate the output file used to calculate energy loss for muons. 

In [ ]:
e_initials = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
e_initials += [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
e_initials += [10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000]

dfs = []
zvals = []
rvals = []
efvals = []
eivals = []

for i,ei in enumerate(e_initials):
    print(ei)

    df_temp = pd.read_csv(f'muonSteps_{ei:d}_GeV.csv')
    
    filter = (df_temp['track']==1)
    z = -1*df_temp[filter]['z(m)'].values
    x = df_temp[filter]['x(m)'].values
    y = df_temp[filter]['y(m)'].values
    e = ei - df_temp[filter]['E(GeV)'].values

    r = np.sqrt(x**2 + y**2)

    efvals += e.tolist()
    rvals += r.tolist()
    zvals += z.tolist()
    eivals += (ei*np.ones_like(e)).tolist()
    
    #if i==0:
    #    df = df_temp.copy
    #else:
    #    df = pd.concatenate(d

zvals = np.array(zvals)
rvals = np.array(rvals)
efvals = np.array(efvals)
eivals = np.array(eivals)

df_dict = {}
df_dict['z'] = zvals
df_dict['r'] = rvals
df_dict['e_initial'] = eivals
df_dict['e'] = efvals


In [ ]:
print(len(efvals))

df = pd.DataFrame.from_dict(df_dict)

In [ ]:
df
df.to_parquet('muons_summary_from_GEANT4_simulations.parquet')

In [ ]:
plt.hist(efvals, bins=1000);

In [ ]:
eivals

In [ ]:
npts = len(zvals)
print(f'{npts = }')

indices = np.arange(0,npts,1, dtype=int)
indices
np.random.shuffle(indices)
print(indices[0:10])
npts_to_plot = 5000
idx = indices[0:npts_to_plot]

plt.plot(zvals[idx],efvals[idx],'.', markersize=0.5)
plt.xlabel(f'z distance')
plt.ylabel(f'energy (GeV)')
;

# Sandbox

Code below this was used for prototyping and experimenting. 


What follows is not directly used in the calculations for eloss. It was some 
prototyping code I used in developing the eloss calculations. 

In [ ]:
#import scikit-learn.

In [ ]:
import bisect


def find_neighbors(sorted_list, x):
    """
    Given sorted_list (ascending) and a value x,
    return (low, high) where:
      - low  = the largest element <= x (or None if x < sorted_list[0])
      - high = the smallest element >= x (or None if x > sorted_list[-1])
    """
    idx = bisect.bisect_left(sorted_list, x)
    # idx is the insertion point to keep the list sorted.
    if idx == 0:
        # x is <= first element
        return None, sorted_list[0]
    elif idx == len(sorted_list):
        # x is greater than all elements
        return sorted_list[-1], None
    else:
        # sorted_list[idx-1] < x <= sorted_list[idx]
        return sorted_list[idx-1], sorted_list[idx]


In [ ]:
# My own 
d = 3000
dwidth = 0.01 * d

print(e_initials)
E_muon = 45040
elo,ehi = find_neighbors(e_initials, E_muon)
delta_e = ehi - elo

print(elo, ehi, delta_e)

frequencies = []
eis = []
efs = []

plt.figure(figsize=(16,8))

# First find the ranges
elo_min = [1e99, 1e99]
ehi_max = [-1e99, -1e99]
for i,Ei in enumerate([elo,ehi]):
    
    filter = (eivals==Ei)
    filter = filter & (zvals > d-dwidth) & (zvals < d+dwidth)
    
    #vals = Ei - efvals[filter]
    vals = efvals[filter]

    if len(vals) == 0:
        print("NO DATA TO WORK WITH")
        continue

    
    test_min = min(vals)
    test_max = max(vals)
    if test_min <= elo_min[i]:
        elo_min[i] = test_min
    if test_max >= ehi_max[i]:
        ehi_max[i] = test_max

for i,Ei in enumerate([elo,ehi]):
    #Ei = 30000
    
    filter = (eivals==Ei)
    filter = filter & (zvals > d-dwidth) & (zvals < d+dwidth)
    
    #vals = Ei - efvals[filter]
    vals = efvals[filter]

    
    plt.subplot(2,2,1)
    plt.hist(vals, bins=100, range=(elo_min[i], ehi_max[i]),label=f'{Ei}', alpha=0.5);
    plt.legend()
    
    kde = stats.gaussian_kde(vals)
    xpts = np.linspace(elo_min[i], ehi_max[i], 100)
    ypts = kde(xpts)
    #frequencies = kde.evaluate(xpts)

    # Shift up the lower one
    #if i==0:
    #    xpts += delta_e

    # Normalize the xpts
    e_range = ehi_max[i] - elo_min[i]
    xpts -= elo_min[i]
    xpts /= e_range
    
    plt.subplot(2,2,2)
    plt.plot(xpts,ypts, label=f'{Ei}')
    #plt.xlim(0,1.1*ehi)

    efs += xpts.tolist()
    eis += (Ei * np.ones_like(xpts)).tolist()
    frequencies += ypts.tolist()
    plt.legend()


data = np.array([eis, efs])
print(data.shape, len(frequencies))
interp = LinearNDInterpolator(data.T, frequencies )

#xpts_temp = np.linspace(elo_min,ehi_max,100)
xpts_temp = np.linspace(0,1,100)

ypts_temp = interp(E_muon,xpts_temp);

filter = ypts_temp==ypts_temp

# Cut out the nans since some of the points are out of range
xpts = xpts_temp[filter]
ypts = ypts_temp[filter]

# Shift the xpoints down
#xpts -= (ehi - E_muon)

#plt.figure()
plt.plot(xpts,ypts, label=f'{E_muon}')
plt.legend()

# Sample points
dx = xpts[1] - xpts[0]
#print(dx)
#print(ypts)

cdf = np.cumsum(ypts)*dx
#print(cdf)

cdf /= cdf[-1]

#print(cdf)

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(xpts, cdf, label='CDF')
#print(cdf)
plt.legend()

#filter = (xpts>=elo_min) & (xpts<=ehi_max)
#print(cdf)
#print(cdf[filter])

spl = CubicSpline(xpts, cdf)

gendata = []
ngendata = 5000
nfail = 0

# Scaling
frac_of_diff = (E_muon - elo) / (ehi - elo)
print(f"{frac_of_diff = }")
e_range_0 = ehi_max[0] - elo_min[0]
e_range_1 = ehi_max[1] - elo_min[1]
print(f"{e_range_0 = }")
print(f"{e_range_1 = }")

e_muon_range = e_range_0 + ((e_range_1 - e_range_0)*frac_of_diff)
e_muon_lo   = elo_min[0] + ((elo_min[1] - elo_min[0])*frac_of_diff)
print(f"{e_muon_range = }")
print(f"{e_muon_lo = }")


for i in range(ngendata):
    u = np.random.random() # Generates a float between 0.0 and 1.0
    ynew = spl.solve(u)
    
    #xnew = max(ynew)
    #filter = (ynew>0) & (ynew<ehi_max)
    filter = (ynew>0) & (ynew<1)

    xnew = ynew[filter]
    if len(xnew) == 1:
        good_val = xnew[0]
        good_val *= e_muon_range
        good_val += e_muon_lo
    else:
        print(u, ynew, xnew)
        nfail += 1
        continue
    #print(good_val)
    #plt.subplot(1,3,2)
    #print(xnew,u)
    #plt.plot(xnew, u, 'ro', markersize=5)
    
    gendata.append(good_val)

plt.subplot(1,2,2)
plt.hist(gendata, bins=200)#, range=(0,1))

print(f'{nfail = }')
;

In [ ]:
#plt.plot(xpts, np.cumsum(ypts))